In [37]:
%cd /content/drive/MyDrive/NLP Made

/content/drive/MyDrive/NLP Made


Вручную проставлены теги на основе столбца "category" - это категория, которую проставляет сам ресторан. И называет ее как угодно


In [38]:
!pip install pymystem3==0.1.10 -q
#!pip install gensim --upgrade -q

In [ ]:
import numpy as np
import pandas as pd
import html
from pymystem3 import Mystem


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def normalize(sentence):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    return ''.join(sentence).strip()

def collect(row, word_to_lemma, count, stemmer, correct_spelling):
    # Заменить html сущности на символы
    row = html.unescape(row)

    # Оставить только символы Аа_Яя Aa_Zz и привести в нижний регистр
    normalized = normalize(row)

    # Разделить предложения на слова по ' ', убрать пустые элементы, заменить опечатки
    list_row = list()
    for word in normalized.split(' '):
        if word != '':
            correct_form = correct_spelling[word]
            # Здесь одно слово может быть разбито на 2, "напримертакое" -> "например такое". Их нужно собрать
            for part in correct_form.split(' '):
                list_row.append(part)

    # Лемматизация слов и сохранение пары слово:лемма в словарь для ускорения работы.
    lemmatized = list()
    for word in list_row:
        if word not in word_to_lemma:
            word_to_lemma[word] = stemmer.lemmatize(word.strip())[0]
        lemma = word_to_lemma[word]

        if lemma in count:
            count[lemma] += 1
        else:
            count[lemma] = 1
        lemmatized.append(lemma)
    
    return ' '.join(lemmatized)

def keep_common(sent, counts, dictionary_threshold, unk_token, keep_order):
    common_words = [unk_token if counts[word] < dictionary_threshold else word for word in sent.split()]

    # Если отсортировать слова, то уберется много дубликатов в корпусе
    if not keep_order:
        common_words = sorted(common_words)
    
    return ' '.join(common_words)

def clean_price(data):
    # Много строк без информации и большой ценой
    data['name_dish'] = data['name_dish'].where(data.name_dish != 'Третий набор в подарок')
    data.dropna(inplace = True)

    # Дороже 100к
    data.price = data.price.apply(lambda x: x/1000 if x>100000 else x)

    # Дороже 15к и не сеты
    idx = data.loc[(data.price > 15000) & ~(data.category.isin(['Рыба и морепродукты','Морепродукты', 'Сеты', 'Комбо', 'Наборы','Торты', 'Мясо']))].index
    data['price'] = data['price'].where(~data.index.isin(idx), data['price']/100)

    # Оставшиеся сеты
    data['price'] = data['price'].apply(lambda x: x/100 if x>35000 else x)

    data = data.astype({'price': int})
    return data

def clean_words(data, dictionary_threshold, unk_token, keep_order, to_spell_path):
    stemmer = Mystem()

    # Словарь с заменой опечаток
    correct_spelling = np.load(to_spell_path, allow_pickle=True).item()
    
    # Нормализация и подсчет встречаемости слов
    word_to_lemma = dict()
    count_description = dict()
    count_name = dict()
    data['product_description'] = data['product_description'].apply(lambda x: collect(x, word_to_lemma, count_description, stemmer, correct_spelling))
    data['name_dish'] = data['name_dish'].apply(lambda x: collect(x, word_to_lemma, count_name, stemmer, correct_spelling))

    # Замена редких слов на <UNK>
    data['product_description'] = data['product_description'].apply(lambda x: keep_common(x, count_description, dictionary_threshold, unk_token, keep_order))
    data['name_dish'] = data['name_dish'].apply(lambda x: keep_common(x, count_name, dictionary_threshold, unk_token, keep_order))

    # Удаление дубликатов
    data.drop_duplicates(subset = ['category','name_dish','product_description','tags_menu'], inplace = True)
    return data

def clean_df(data, dictionary_threshold = 30, unk_token = '<UNK>', keep_order = True, to_spell_path = 'to_spell.npy'):
    data = data.copy()

    # Замена словаря в поле tags_menu на его значение
    data['tags_menu'] = data['tags_menu'].apply(lambda x: [k for k in x.keys()][0])

    # Удаление одной строки с пустыми name_dish, product_description, price
    data.dropna(inplace = True)
    data.reset_index(drop=True, inplace=True)

    # Очистка price
    data = clean_price(data)
    
    # Очистка name_dish, product_description
    data = clean_words(data, dictionary_threshold, unk_token, keep_order, to_spell_path)

    return data

In [39]:
import pandas as pd
pd.set_option('display.max_rows', 500)

data = pd.read_pickle('menu_df_for_made.pkl')

In [40]:
from preprocess import clean_df
data_c = clean_df(data, dictionary_threshold=5, unk_token='', keep_order = True)

In [ ]:
eng_words = {k: None for k,v in w_t_l.items() if any([x in list(map(chr, range(ord('a'), ord('z') + 1))) for x in k])}

In [ ]:
%cd /content/drive/MyDrive/NLP Made

/content/drive/MyDrive/NLP Made


In [ ]:
!pip install googletrans==3.1.0a0 -q

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 1.3 MB 14.7 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 


In [ ]:
!python3 translator_threads.py

# Bert finetuning

In [41]:
data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']

In [46]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

data_c['y'] = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
data_c = data_c.astype({'y': int})

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_c['X'], data_c['y'], test_size=0.2, stratify = data_c['tags_menu'], random_state=42, shuffle = True)

In [49]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, random_state=42, shuffle = True)

In [50]:
pd.DataFrame({'X':X_train, 'y': y_train}).to_csv('train.csv')
pd.DataFrame({'X':X_valid, 'y': y_valid}).to_csv('valid.csv')
pd.DataFrame({'X':X_test, 'y': y_test}).to_csv('test.csv')

# Naive bayes

In [ ]:
data_c

,product_id,category,name_dish,product_description,price,tags_menu
0,300146805,Пицца,милан пицца фирменный,болгарский в вырезка гриб клюквенный мариноват...,620,пицца
1,300146840,Пицца,пицца фермерский,ветчина говяжий гриб грудка индейка корнишон к...,650,пицца
2,300146870,Пицца,пицца сезон четыре,болгарский гриб лук маслина моцарелл оливка пе...,550,пицца
3,300146918,Пицца,деревенский пицца,бекон ветчина гриб индейка лук моцарелл пицца ...,620,пицца
4,300146996,Пицца,маргарита пицца,моцарелл пицца соус сыр томатный,490,пицца
...,...,...,...,...,...,...
2050004,331415633,Драники,драник и картофельный лосось слабосоленый слив...,,530,драники
2050005,331415634,Драники,драник из свекла,,195,драники
2050006,331415635,Драники,драник из капуста,,195,драники
2050013,333962918,Драники,драник картофельный,шт,139,драники


In [ ]:
data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

enc = OrdinalEncoder()

y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(data_c['X'].values)

N_SPLITS = 5
skf = StratifiedKFold(n_splits = N_SPLITS)

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    model = MultinomialNB()
    model.fit(X[train_index, :], y[train_index,:])

    y_pred = model.predict(X[test_index,:])
    y_true = y[test_index,:]

    if i == 0:
        report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
    else:
        report += pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T

report = report / N_SPLITS

In [ ]:
#CountVectorizer()@100
report

,precision,recall,f1-score,support
азиатский суп,0.098380,0.144444,0.113813,18.000000
блины и оладьи,0.769784,0.885111,0.823139,1396.200000
бургер,0.874415,0.932144,0.902253,6334.000000
вегетарианская еда,0.242173,0.246549,0.243001,912.600000
вок и лапша,0.843300,0.944623,0.891075,5626.800000
гарниры,0.800228,0.751307,0.774906,5393.000000
драники,0.394853,0.193103,0.251751,29.000000
закуски,0.618536,0.483897,0.542289,4179.400000
каша и гранола,0.550259,0.581250,0.563168,64.000000
кофе,0.258750,0.651530,0.320943,443.600000


In [ ]:
#CountVectorizer()@50
report

,precision,recall,f1-score,support
азиатский суп,0.120606,0.077778,0.087576,18.000000
блины и оладьи,0.763271,0.882502,0.818231,1397.400000
бургер,0.872067,0.933382,0.901590,6346.600000
вегетарианская еда,0.253324,0.244858,0.247780,914.000000
вок и лапша,0.842521,0.946795,0.891607,5627.200000
гарниры,0.801519,0.754969,0.777465,5411.600000
драники,0.288095,0.110345,0.158450,29.000000
закуски,0.618458,0.495971,0.549566,4194.200000
каша и гранола,0.603414,0.512500,0.551080,64.000000
кофе,0.261130,0.625711,0.317608,448.800000


In [ ]:
#CountVectorizer()@10
report

,precision,recall,f1-score,support
азиатский суп,0.000000,0.000000,0.000000,18.000000
блины и оладьи,0.770012,0.867096,0.815319,1398.000000
бургер,0.866042,0.937072,0.900069,6362.800000
вегетарианская еда,0.298028,0.220643,0.252348,916.400000
вок и лапша,0.836864,0.949603,0.889663,5627.400000
гарниры,0.799323,0.762670,0.780516,5438.000000
драники,0.200000,0.006897,0.013333,29.000000
закуски,0.611940,0.516011,0.558889,4209.600000
каша и гранола,0.775935,0.355913,0.487654,64.600000
кофе,0.274400,0.511314,0.302423,459.600000


In [ ]:
#CountVectorizer()
report

,precision,recall,f1-score,support
азиатский суп,0.128571,0.044444,0.061364,18.000000
блины и оладьи,0.764817,0.878081,0.817184,1397.600000
бургер,0.870929,0.934538,0.901520,6354.800000
вегетарианская еда,0.269522,0.244432,0.255089,915.600000
вок и лапша,0.840340,0.947864,0.890859,5627.400000
гарниры,0.801811,0.757699,0.779060,5421.400000
драники,0.192424,0.062069,0.093050,29.000000
закуски,0.617268,0.503761,0.553835,4200.800000
каша и гранола,0.648593,0.468942,0.542504,64.400000
кофе,0.267090,0.597046,0.315865,454.200000


In [ ]:
# TfidfVectorizer
report

,precision,recall,f1-score,support
азиатский суп,0.000000,0.000000,0.000000,18.000000
блины и оладьи,0.847408,0.736265,0.787297,1397.600000
бургер,0.860016,0.928306,0.892781,6354.800000
вегетарианская еда,0.579891,0.097638,0.166424,915.600000
вок и лапша,0.858697,0.938232,0.896701,5627.400000
гарниры,0.791114,0.730437,0.759468,5421.400000
драники,0.000000,0.000000,0.000000,29.000000
закуски,0.629543,0.412588,0.498001,4200.800000
каша и гранола,0.400000,0.009327,0.018182,64.400000
кофе,0.290234,0.036994,0.062953,454.200000


# LogReg tf-idf

In [ ]:
data_c

,product_id,category,name_dish,product_description,price,tags_menu
0,300146805,Пицца,милан фирменный пицца,болгарский перец гриб помидор свиной вырезка м...,620,пицца
1,300146840,Пицца,фермерский пицца,пицца соус сливочный ветчина индейка говяжий я...,650,пицца
2,300146870,Пицца,пицца четыре сезон,болгарский перец гриб лук маслина оливка помид...,550,пицца
3,300146918,Пицца,деревенский пицца,бекон ветчина индейка гриб лук помидор салями ...,620,пицца
4,300146996,Пицца,пицца маргарита,пицца соус томатный сыр моцарелл,490,пицца
...,...,...,...,...,...,...
2050004,331415633,Драники,картофельный драник со слабосоленый лосось и с...,,530,драники
2050005,331415634,Драники,драник из свекла,,195,драники
2050006,331415635,Драники,драник из капуста,,195,драники
2050013,333962918,Драники,драник картофельный,шт,139,драники


In [ ]:
data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

enc = OrdinalEncoder()

y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))

vectorizer = TfidfVectorizer(ngram_range = (1,1), min_df = 5, max_features=12000, )
X = vectorizer.fit_transform(data_c['X'].values)

N_SPLITS = 5
skf = StratifiedKFold(n_splits = N_SPLITS)

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    model = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=-1)
    model.fit(X[train_index, :], y[train_index,:])

    y_pred = model.predict(X[test_index,:])
    y_true = y[test_index,:]

    if i == 0:
        report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
    else:
        report += pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T

    break

In [ ]:
# TfidfVectorizer(ngram_range = (1,1), min_df = 5, max_features=12000, )
report

,precision,recall,f1-score,support
азиатский суп,0.166667,0.055556,0.083333,18.000000
блины и оладьи,0.939850,0.885897,0.912076,1411.000000
бургер,0.953807,0.924493,0.938921,6410.000000
вегетарианская еда,0.426087,0.212581,0.283647,922.000000
вок и лапша,0.889251,0.914694,0.901793,5662.000000
гарниры,0.801679,0.826380,0.813842,5777.000000
драники,0.450000,0.620690,0.521739,29.000000
закуски,0.622103,0.655553,0.638390,4259.000000
каша и гранола,0.491525,0.446154,0.467742,65.000000
кофе,0.500000,0.019231,0.037037,468.000000


In [ ]:
import eli5
eli5.show_weights(estimator=model, 
                  feature_names= list(vectorizer.get_feature_names()),
                  target_names = enc.categories_[0],
                 top=(20, 20))

# LogReg fasttext

In [ ]:
NUM_LABELS = 39

In [ ]:
!pip install gensim -U

     |████████████████████████████████| 24.1 MB 2.8 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
import numpy as np
import torch
from torch import nn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']
data_c['X_len'] = data_c['X'].apply(lambda x: len([w for w in x.split(' ') if w != '']))
data_c = data_c.loc[data_c['X_len'] > 0]

scaler = StandardScaler()
data_c['price'] = scaler.fit_transform(data_c['price'].values.reshape(-1,1))

enc = OrdinalEncoder()
y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
data_c['y'] = y
data_c = data_c.astype({'y':int})

In [ ]:
word_to_idx = dict()
c = 0

for row in data_c['X']:
    for word in row.split():
        if word not in word_to_idx:
            word_to_idx[word] = c
            c += 1

idx_to_word = {v: k for k, v in word_to_idx.items()}

In [ ]:
data_c['X_idx'] = data_c['X'].apply(lambda x: [word_to_idx[word] for word in x.split(' ') if word != ''])

In [ ]:
ft_model = gensim.models.fasttext.FastTextKeyedVectors.load('Fasttext/model.model')

In [ ]:
vectors = [0] * len(idx_to_word)

for idx, word in idx_to_word.items():
    vectors[idx] = ft_model.get_vector(word)

In [ ]:
PAD_VEC = np.array([0.0 for _ in vectors[0]])
PAD_IDX = len(vectors)
vectors.append(PAD_VEC)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class CustomTextDataset(Dataset):
    def __init__(self, price, text, seq_len, labels):
        self.labels = labels
        self.text = text
        self.price = price
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        price = self.price[idx]
        seq_len = self.seq_len[idx]
        return price, text, seq_len, label

def collate_fn(data):
    label_list, price_list, text_list, len_list = list(), list(), list(), list()
    for price, text, seq_len, label in data:
        label_list.append(label)
        price_list.append(price)
        text_list.append(torch.LongTensor(text))
        len_list.append(seq_len)

    label = torch.LongTensor(label_list)
    price = torch.FloatTensor(price_list)
    text = pad_sequence(text_list, padding_value=PAD_IDX, batch_first=True)
    seq_len = torch.LongTensor(len_list)
    return price, text, seq_len, label

train_dataset = CustomTextDataset(X_train['price'].values, X_train['X_idx'].values, X_train['X_len'].values, y_train.values)
test_dataset = CustomTextDataset(X_test['price'].values, X_test['X_idx'].values, X_test['X_len'].values, y_test.values)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=5000, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=5000, shuffle=True, collate_fn=collate_fn)

In [ ]:
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vectors), freeze = True)
        self.fc = nn.Linear(300 + 1, NUM_LABELS)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, price, text, seq_len):
        embedded = self.embedding(text) # batch_len, seq_len, emb_shape
        input = torch.sum(embedded, dim = 1) # batch_len, emb_shape
        input = torch.div(input.permute([1, 0]), seq_len).permute([1, 0]) # batch_len, emb_shape
        input_price = torch.cat((input, price.unsqueeze(1)), dim = 1) #batch_len, emb_shape + 1
        input = self.fc(input_price) # batch_len, NUM_LABELS
        return input

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextModel().to(device)
#for price, text, seq_len, label in train_dataloader:
    #model(price, text, seq_len)
    #break

In [ ]:
import torch.optim as optim

class_sample_count = np.unique(data_c['y'].values, return_counts=True)[1]
weight = torch.FloatTensor(1. / class_sample_count)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, device):
    for epoch in range(20):
        print(f"Epoch {epoch}")
        with torch.no_grad():
            eval_loss = list()
            for price, text, seq_len, label in test_dataloader:
                price = price.to(device)
                text = text.to(device)
                seq_len = seq_len.to(device)

                out = model(price, text, seq_len)
                loss = criterion(out.cpu(), label)
                eval_loss.append(loss.item())
            print(f"Eval loss: {np.mean(eval_loss)}")

        epoch_loss = list()
        for price, text, seq_len, label in train_dataloader:
            price = price.to(device)
            text = text.to(device)
            seq_len = seq_len.to(device)

            optimizer.zero_grad()
            out = model(price, text, seq_len)

            #calculate the loss
            loss = criterion(out.cpu(), label)
            
            #track batch loss
            epoch_loss.append(loss.item())
            
            #backpropagation
            loss.backward()
            
            #update the parameters
            optimizer.step()
        print(f"Train loss: {np.mean(epoch_loss)}")

train(model, device)

Epoch 0
Eval loss: 3.610125968622607
Train loss: 2.2846504478059577
Epoch 1
Eval loss: 1.781737626985062
Train loss: 1.5497588088526528
Epoch 2
Eval loss: 1.362334545268569
Train loss: 1.2358828056493454
Epoch 3
Eval loss: 1.1277903484743694
Train loss: 1.047444889531333
Epoch 4
Eval loss: 0.9772300512291664
Train loss: 0.9210676758952395
Epoch 5
Eval loss: 0.8711702102838561
Train loss: 0.8311938520013933
Epoch 6
Eval loss: 0.7933100087698116
Train loss: 0.7638418275929062
Epoch 7
Eval loss: 0.7348279190617938
Train loss: 0.7123704298713504
Epoch 8
Eval loss: 0.6885601947473925
Train loss: 0.6713577142128577
Epoch 9
Eval loss: 0.6535301125326822
Train loss: 0.6381299199437248
Epoch 10
Eval loss: 0.6219314946684726
Train loss: 0.6108163870297946
Epoch 11
Eval loss: 0.5981995435648186
Train loss: 0.5875728638214472
Epoch 12
Eval loss: 0.576340280299963
Train loss: 0.5679779451274307
Epoch 13
Eval loss: 0.5576027520867282
Train loss: 0.5507770688576106
Epoch 14
Eval loss: 0.5431063785109

In [ ]:
with torch.no_grad():
    y_pred = list()
    y_true = list()
    for price, text, seq_len, label in test_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)

        out = model(price, text, seq_len)
        y_pred.extend(torch.argmax(out.cpu(), dim = 1).tolist())
        y_true.extend(label)

In [ ]:
from sklearn.metrics import classification_report

report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# report no weights
report.astype({'support': int})

,precision,recall,f1-score,support
азиатский суп,0.000000,0.000000,0.000000,18
блины и оладьи,0.885508,0.784954,0.832205,1409
бургер,0.891417,0.888906,0.890160,6391
вегетарианская еда,0.540541,0.043431,0.080402,921
вок и лапша,0.866203,0.902314,0.883890,5661
гарниры,0.770196,0.778763,0.774456,5754
драники,0.000000,0.000000,0.000000,29
закуски,0.531379,0.363186,0.431471,4243
каша и гранола,0.833333,0.076923,0.140845,65
кофе,0.357143,0.032538,0.059642,461


In [ ]:
# report with weights
report.astype({'support': int})

,precision,recall,f1-score,support
азиатский суп,0.045455,0.777778,0.085890,18
блины и оладьи,0.863144,0.904187,0.883189,1409
бургер,0.914561,0.906118,0.910320,6391
вегетарианская еда,0.088470,0.397394,0.144721,921
вок и лапша,0.874764,0.899488,0.886953,5661
гарниры,0.842215,0.703163,0.766433,5754
драники,0.333333,0.896552,0.485981,29
закуски,0.463058,0.381098,0.418100,4243
каша и гранола,0.197719,0.800000,0.317073,65
кофе,0.162605,0.882863,0.274629,461


# LSTM

In [ ]:
!pip install gensim -U -q

     |████████████████████████████████| 24.1 MB 2.8 kB/s 


In [ ]:
import gensim
import numpy as np
import torch
import torch.optim as optim
from torch import nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
NUM_LABELS = 39

In [ ]:
def preprocess_for_nn(data):
    data_c = data.copy()
    data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']
    data_c['X_len'] = data_c['X'].apply(lambda x: len([w for w in x.split(' ') if w != '']))
    data_c = data_c.loc[data_c['X_len'] > 0]

    scaler = StandardScaler()
    data_c['price'] = scaler.fit_transform(data_c['price'].values.reshape(-1,1))

    enc = OrdinalEncoder()
    y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
    data_c['y'] = y
    data_c = data_c.astype({'y':int})
    return data_c, enc

def get_vectors(data):
    data_c = data.copy()
    word_to_idx = dict()
    c = 0

    for row in data_c['X']:
        for word in row.split():
            if word not in word_to_idx:
                word_to_idx[word] = c
                c += 1

    idx_to_word = {v: k for k, v in word_to_idx.items()}

    data_c['X_idx'] = data_c['X'].apply(lambda x: [word_to_idx[word] for word in x.split(' ') if word != ''])

    ft_model = gensim.models.fasttext.FastTextKeyedVectors.load('Fasttext/model.model')

    vectors = [0] * len(idx_to_word)

    for idx, word in idx_to_word.items():
        vectors[idx] = ft_model.get_vector(word)

    PAD_VEC = np.array([0.0 for _ in vectors[0]])
    PAD_IDX = len(vectors)
    vectors.append(PAD_VEC)
    return data_c, word_to_idx, idx_to_word, vectors, PAD_IDX

data_c, enc = preprocess_for_nn(data_c)
data_c, word_to_idx, idx_to_word, vectors, PAD_IDX = get_vectors(data_c)
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
data_c = data_c.loc[data_c.X_len<40]
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, price, text, seq_len, labels):
        self.labels = labels
        self.text = text
        self.price = price
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        price = self.price[idx]
        seq_len = self.seq_len[idx]
        return price, text, seq_len, label

def collate_fn(data):
    label_list, price_list, text_list, len_list = list(), list(), list(), list()
    for price, text, seq_len, label in data:
        label_list.append(label)
        price_list.append(price)
        text_list.append(torch.LongTensor(text))
        len_list.append(seq_len)

    label = torch.LongTensor(label_list)
    price = torch.FloatTensor(price_list)
    text = pad_sequence(text_list, padding_value=PAD_IDX, batch_first=True)
    seq_len = torch.LongTensor(len_list)
    return price, text, seq_len, label

train_dataset = CustomTextDataset(X_train['price'].values, X_train['X_idx'].values, X_train['X_len'].values, y_train.values)
test_dataset = CustomTextDataset(X_test['price'].values, X_test['X_idx'].values, X_test['X_len'].values, y_test.values)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)

In [ ]:
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vectors), freeze = True)
        self.rnn = nn.LSTM(input_size=300, hidden_size=300, batch_first = True, bidirectional = True)
        self.fc1 = nn.Linear(600 + 1, 256)
        self.fc2 = nn.Linear(256, NUM_LABELS)

    def forward(self, price, text, seq_len):
        embedded = self.embedding(text) # batch_len, seq_len, emb_shape
        output, (hidden, cell) = self.rnn(embedded)
        output = torch.cat([hidden[0,:, :], hidden[1,:,:]], dim=1) #batch_len, emb_shape*2
        output = torch.cat((output, price.unsqueeze(1)), dim = 1) #batch_len, emb_shape*2+1
        output = nn.functional.relu(self.fc1(output))
        output = self.fc2(output)
        return output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextModel().to(device)
#with torch.no_grad():
#    for price, text, seq_len, label in train_dataloader:
#        model(price, text, seq_len)
#        break

In [ ]:
import torch.optim as optim

class_sample_count = np.unique(data_c['y'].values, return_counts=True)[1]
weight = torch.FloatTensor(1. / class_sample_count)
weight = np.log(weight*1000000)/100

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
model.load_state_dict(torch.load('LSTM_10_model_unfreeze_emb'), strict=False)

<All keys matched successfully>

In [ ]:
def train(model, device):
    eval_loss_max = 0.16240031869641922
    for epoch in range(10):
        print(f"Epoch {epoch}")
        epoch_loss = list()
        for i, (price, text, seq_len, label) in enumerate(train_dataloader):

            if i % 400 == 0:
                with torch.no_grad():
                    eval_loss = list()
                    for price, text, seq_len, label in test_dataloader:
                        price = price.to(device)
                        text = text.to(device)
                        seq_len = seq_len.to(device)

                        out = model(price, text, seq_len)
                        loss = criterion(out.cpu(), label)
                        eval_loss.append(loss.item())
                    mean_loss = np.mean(eval_loss)
                    if eval_loss_max > mean_loss:
                        eval_loss_max = mean_loss
                        torch.save(model.state_dict(), './LSTM_10_model_unfreeze_emb')
                    print(f"Eval loss: {np.mean(eval_loss)}")
            
            price = price.to(device)
            text = text.to(device)
            seq_len = seq_len.to(device)

            optimizer.zero_grad()
            out = model(price, text, seq_len)

            #calculate the loss
            loss = criterion(out.cpu(), label)
            
            #track batch loss
            epoch_loss.append(loss.item())
            
            #backpropagation
            loss.backward()
            
            #update the parameters
            optimizer.step()
        print(f"Train loss: {np.mean(epoch_loss)}")

train(model, device)

In [ ]:
# 4 эпохи + разморозка ембеддингов и уменьшение lr в 2 раза
# 10 words
with torch.no_grad():
    y_pred = list()
    y_true = list()
    for price, text, seq_len, label in test_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)

        out = model(price, text, seq_len)
        y_pred.extend(torch.argmax(out.cpu(), dim = 1).tolist())
        y_true.extend(label)

report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
report.astype({'support': int})

,precision,recall,f1-score,support
азиатский суп,0.631579,0.705882,0.666667,17
блины и оладьи,0.948252,0.969264,0.958643,1399
бургер,0.965189,0.954422,0.959776,6275
вегетарианская еда,0.733154,0.301887,0.427673,901
вок и лапша,0.932732,0.960886,0.946600,5599
гарниры,0.871887,0.887635,0.879690,5758
драники,0.764706,0.448276,0.565217,29
закуски,0.736939,0.753130,0.744947,4233
каша и гранола,0.760000,0.876923,0.814286,65
кофе,0.446301,0.405640,0.425000,461


In [ ]:
# 30 words
with torch.no_grad():
    y_pred = list()
    y_true = list()
    for price, text, seq_len, label in test_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)

        out = model(price, text, seq_len)
        y_pred.extend(torch.argmax(out.cpu(), dim = 1).tolist())
        y_true.extend(label)

report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
report.astype({'support': int})

,precision,recall,f1-score,support
азиатский суп,0.250000,0.125000,0.166667,16
блины и оладьи,0.943732,0.959450,0.951526,1381
бургер,0.973717,0.943428,0.958333,5851
вегетарианская еда,0.580897,0.342923,0.431259,869
вок и лапша,0.934794,0.958403,0.946451,5385
гарниры,0.883989,0.875768,0.879859,5699
драники,0.638889,0.793103,0.707692,29
закуски,0.721296,0.740955,0.730993,4146
каша и гранола,0.771429,0.843750,0.805970,64
кофе,0.435252,0.274376,0.336579,441


# GRU

In [ ]:
!pip install gensim -U -q

     |████████████████████████████████| 24.1 MB 2.8 kB/s 


In [ ]:
import gensim
import numpy as np
import torch
import torch.optim as optim
from torch import nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
NUM_LABELS = 39

In [ ]:
def preprocess_for_nn(data):
    data_c = data.copy()
    data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']
    data_c['X_len'] = data_c['X'].apply(lambda x: len([w for w in x.split(' ') if w != '']))
    data_c = data_c.loc[data_c['X_len'] > 0]

    scaler = StandardScaler()
    data_c['price'] = scaler.fit_transform(data_c['price'].values.reshape(-1,1))

    enc = OrdinalEncoder()
    y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
    data_c['y'] = y
    data_c = data_c.astype({'y':int})
    return data_c, enc

def get_vectors(data):
    data_c = data.copy()
    word_to_idx = dict()
    c = 0

    for row in data_c['X']:
        for word in row.split():
            if word not in word_to_idx:
                word_to_idx[word] = c
                c += 1

    idx_to_word = {v: k for k, v in word_to_idx.items()}

    data_c['X_idx'] = data_c['X'].apply(lambda x: [word_to_idx[word] for word in x.split(' ') if word != ''])

    ft_model = gensim.models.fasttext.FastTextKeyedVectors.load('Fasttext/model.model')

    vectors = [0] * len(idx_to_word)

    for idx, word in idx_to_word.items():
        vectors[idx] = ft_model.get_vector(word)

    PAD_VEC = np.array([0.0 for _ in vectors[0]])
    PAD_IDX = len(vectors)
    vectors.append(PAD_VEC)
    return data_c, word_to_idx, idx_to_word, vectors, PAD_IDX

data_c, enc = preprocess_for_nn(data_c)
data_c, word_to_idx, idx_to_word, vectors, PAD_IDX = get_vectors(data_c)
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
data_c = data_c.loc[data_c.X_len<30]
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, price, text, seq_len, labels):
        self.labels = labels
        self.text = text
        self.price = price
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        price = self.price[idx]
        seq_len = self.seq_len[idx]
        return price, text, seq_len, label

def collate_fn(data):
    label_list, price_list, text_list, len_list = list(), list(), list(), list()
    for price, text, seq_len, label in data:
        label_list.append(label)
        price_list.append(price)
        text_list.append(torch.LongTensor(text))
        len_list.append(seq_len)

    label = torch.LongTensor(label_list)
    price = torch.FloatTensor(price_list)
    text = pad_sequence(text_list, padding_value=PAD_IDX, batch_first=True)
    seq_len = torch.LongTensor(len_list)
    return price, text, seq_len, label

train_dataset = CustomTextDataset(X_train['price'].values, X_train['X_idx'].values, X_train['X_len'].values, y_train.values)
test_dataset = CustomTextDataset(X_test['price'].values, X_test['X_idx'].values, X_test['X_len'].values, y_test.values)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)

In [ ]:
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vectors), freeze = True)
        self.rnn = nn.GRU(input_size=300, hidden_size=300, batch_first = True, bidirectional = True)
        self.fc1 = nn.Linear(600 + 1, 256)
        self.fc2 = nn.Linear(256, NUM_LABELS)

    def forward(self, price, text, seq_len):
        embedded = self.embedding(text) # batch_len, seq_len, emb_shape
        output, hidden = self.rnn(embedded)
        output = torch.cat([hidden[0,:, :], hidden[1,:,:]], dim=1) #batch_len, emb_shape*2
        output = torch.cat((output, price.unsqueeze(1)), dim = 1) #batch_len, emb_shape*2+1
        output = nn.functional.relu(self.fc1(output))
        output = self.fc2(output)
        return output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextModel().to(device)
with torch.no_grad():
    for price, text, seq_len, label in train_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)
        model(price, text, seq_len)
        break

In [ ]:
import torch.optim as optim

class_sample_count = np.unique(data_c['y'].values, return_counts=True)[1]
weight = torch.FloatTensor(1. / class_sample_count)
weight = np.log(weight*1000000)/100

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, device):
    eval_loss_max = 0.2
    for epoch in range(10):
        print(f"Epoch {epoch}")
        epoch_loss = list()
        for i, (price, text, seq_len, label) in enumerate(train_dataloader):

            if i % 400 == 0:
                with torch.no_grad():
                    eval_loss = list()
                    for price, text, seq_len, label in test_dataloader:
                        price = price.to(device)
                        text = text.to(device)
                        seq_len = seq_len.to(device)

                        out = model(price, text, seq_len)
                        loss = criterion(out.cpu(), label)
                        eval_loss.append(loss.item())
                    mean_loss = np.mean(eval_loss)
                    if eval_loss_max > mean_loss:
                        eval_loss_max = mean_loss
                        torch.save(model.state_dict(), './GRU_30')
                    print(f"Eval loss: {np.mean(eval_loss)}")
            
            price = price.to(device)
            text = text.to(device)
            seq_len = seq_len.to(device)

            optimizer.zero_grad()
            out = model(price, text, seq_len)

            #calculate the loss
            loss = criterion(out.cpu(), label)
            
            #track batch loss
            epoch_loss.append(loss.item())
            
            #backpropagation
            loss.backward()
            
            #update the parameters
            optimizer.step()
        print(f"Train loss: {np.mean(epoch_loss)}")

train(model, device)

Epoch 0
Eval loss: 3.6792612262343427
Eval loss: 0.29152624429292373
Eval loss: 0.25422492306360694
Eval loss: 0.22982598502361978
Eval loss: 0.21928413889956067
Train loss: 0.3303072519253853
Epoch 1
Eval loss: 0.21797809611395694
Eval loss: 0.2109480094355884
Eval loss: 0.20165191578034375
Eval loss: 0.19781058589520256
Eval loss: 0.1923676671273551
Train loss: 0.20021503043419145
Epoch 2
Eval loss: 0.19644661577450326
Eval loss: 0.19032416851567172
Eval loss: 0.18700966213472314
Eval loss: 0.18843786939432103
Eval loss: 0.18103033888966646
Train loss: 0.17593201958874155
Epoch 3
Eval loss: 0.18234120609926419
Eval loss: 0.18081746627474182
Eval loss: 0.17618767119270082
Eval loss: 0.1775358840458842
Eval loss: 0.1747961846933971
Train loss: 0.1558462228247556
Epoch 4
Eval loss: 0.17387965064687075
Eval loss: 0.17856438893342078
Eval loss: 0.1767528853345034
Eval loss: 0.1750946183914369
Eval loss: 0.17155427551284105
Train loss: 0.13736325078569764
Epoch 5
Eval loss: 0.1722059706021

KeyboardInterrupt: ignored

In [ ]:
# 30 words GRU
model.load_state_dict(torch.load('GRU_30'))
with torch.no_grad():
    y_pred = list()
    y_true = list()
    for price, text, seq_len, label in test_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)

        out = model(price, text, seq_len)
        y_pred.extend(torch.argmax(out.cpu(), dim = 1).tolist())
        y_true.extend(label)

report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
report.astype({'support': int})

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
азиатский суп,0.666667,0.125000,0.210526,16
блины и оладьи,0.963757,0.943519,0.953531,1381
бургер,0.974120,0.945650,0.959674,5851
вегетарианская еда,0.571429,0.382048,0.457931,869
вок и лапша,0.934299,0.958589,0.946288,5385
гарниры,0.891167,0.869275,0.880085,5699
драники,0.625000,0.689655,0.655738,29
закуски,0.689961,0.769175,0.727418,4146
каша и гранола,0.840909,0.578125,0.685185,64
кофе,0.778947,0.167800,0.276119,441


# CNN

In [ ]:
!pip install gensim -U -q

     |████████████████████████████████| 24.1 MB 2.8 kB/s 


In [ ]:
import gensim
import numpy as np
import torch
import torch.optim as optim
from torch import nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
NUM_LABELS = 39

In [ ]:
def preprocess_for_nn(data):
    data_c = data.copy()
    data_c['X'] = data_c['name_dish'] + ' ' + data_c['product_description']
    data_c['X_len'] = data_c['X'].apply(lambda x: len([w for w in x.split(' ') if w != '']))
    data_c = data_c.loc[data_c['X_len'] > 0]

    scaler = StandardScaler()
    data_c['price'] = scaler.fit_transform(data_c['price'].values.reshape(-1,1))

    enc = OrdinalEncoder()
    y = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
    data_c['y'] = y
    data_c = data_c.astype({'y':int})
    return data_c, enc

def get_vectors(data):
    data_c = data.copy()
    word_to_idx = dict()
    c = 0

    for row in data_c['X']:
        for word in row.split():
            if word not in word_to_idx:
                word_to_idx[word] = c
                c += 1

    idx_to_word = {v: k for k, v in word_to_idx.items()}

    data_c['X_idx'] = data_c['X'].apply(lambda x: [word_to_idx[word] for word in x.split(' ') if word != ''])

    ft_model = gensim.models.fasttext.FastTextKeyedVectors.load('Fasttext/model.model')

    vectors = [0] * len(idx_to_word)

    for idx, word in idx_to_word.items():
        vectors[idx] = ft_model.get_vector(word)

    PAD_VEC = np.array([0.0 for _ in vectors[0]])
    PAD_IDX = len(vectors)
    vectors.append(PAD_VEC)
    return data_c, word_to_idx, idx_to_word, vectors, PAD_IDX

data_c, enc = preprocess_for_nn(data_c)
data_c, word_to_idx, idx_to_word, vectors, PAD_IDX = get_vectors(data_c)
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
data_c = data_c.loc[data_c.X_len<30]
X_train, X_test, y_train, y_test = train_test_split(data_c.drop('y', axis = 1), data_c['y'], test_size=0.2, random_state=42, stratify = data_c['y'])

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, price, text, seq_len, labels):
        self.labels = labels
        self.text = text
        self.price = price
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        price = self.price[idx]
        seq_len = self.seq_len[idx]
        return price, text, seq_len, label

def collate_fn(data):
    label_list, price_list, text_list, len_list = list(), list(), list(), list()
    for price, text, seq_len, label in data:
        label_list.append(label)
        price_list.append(price)
        text_list.append(torch.LongTensor(text))
        len_list.append(seq_len)

    label = torch.LongTensor(label_list)
    price = torch.FloatTensor(price_list)
    text = pad_sequence(text_list, padding_value=PAD_IDX, batch_first=True)
    seq_len = torch.LongTensor(len_list)
    return price, text, seq_len, label

train_dataset = CustomTextDataset(X_train['price'].values, X_train['X_idx'].values, X_train['X_len'].values, y_train.values)
test_dataset = CustomTextDataset(X_test['price'].values, X_test['X_idx'].values, X_test['X_len'].values, y_test.values)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=500, shuffle=True, collate_fn=collate_fn)

In [ ]:
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vectors), freeze = True)
        self.cnn1 = nn.Conv1d(300, 200, 1,)
        self.cnn2 = nn.Conv1d(300, 200, 2)
        self.cnn3 = nn.Conv1d(300, 200, 3)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(600 + 1, 256)
        self.fc2 = nn.Linear(256, NUM_LABELS)

    def forward(self, price, text, seq_len):
        embedded = self.embedding(text) # batch_len, seq_len, emb_shape
        embedded = embedded.permute(0, 2, 1) # batch_len, emb_shape, seq_len
        output1 = self.pool(self.cnn1(embedded))
        output2 = self.pool(self.cnn2(embedded))
        output3 = self.pool(self.cnn3(embedded)) # batch_len, 200, 1
        output = torch.cat([output1, output2, output3], dim=1).squeeze(-1) #batch_len, emb_shape*2
        output = torch.cat((output, price.unsqueeze(1)), dim = 1) #batch_len, emb_shape*2+1
        output = nn.functional.relu(self.fc1(output))
        output = self.fc2(output)
        return output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextModel().to(device)
with torch.no_grad():
    for price, text, seq_len, label in train_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)
        model(price, text, seq_len)
        break

In [ ]:
import torch.optim as optim

class_sample_count = np.unique(data_c['y'].values, return_counts=True)[1]
weight = torch.FloatTensor(1. / class_sample_count)
weight = np.log(weight*1000000)/100

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, device):
    eval_loss_max = 0.2
    for epoch in range(10):
        print(f"Epoch {epoch}")
        epoch_loss = list()
        for i, (price, text, seq_len, label) in enumerate(train_dataloader):

            if i % 400 == 0:
                with torch.no_grad():
                    eval_loss = list()
                    for price, text, seq_len, label in test_dataloader:
                        price = price.to(device)
                        text = text.to(device)
                        seq_len = seq_len.to(device)

                        out = model(price, text, seq_len)
                        loss = criterion(out.cpu(), label)
                        eval_loss.append(loss.item())
                    mean_loss = np.mean(eval_loss)
                    if eval_loss_max > mean_loss:
                        eval_loss_max = mean_loss
                        torch.save(model.state_dict(), './CNN_30')
                    print(f"Eval loss: {np.mean(eval_loss)}")
            
            price = price.to(device)
            text = text.to(device)
            seq_len = seq_len.to(device)

            optimizer.zero_grad()
            out = model(price, text, seq_len)

            #calculate the loss
            loss = criterion(out.cpu(), label)
            
            #track batch loss
            epoch_loss.append(loss.item())
            
            #backpropagation
            loss.backward()
            
            #update the parameters
            optimizer.step()
        print(f"Train loss: {np.mean(epoch_loss)}")

train(model, device)

In [ ]:
# 30 words CNN
model.load_state_dict(torch.load('CNN_30'))
with torch.no_grad():
    y_pred = list()
    y_true = list()
    for price, text, seq_len, label in test_dataloader:
        price = price.to(device)
        text = text.to(device)
        seq_len = seq_len.to(device)

        out = model(price, text, seq_len)
        y_pred.extend(torch.argmax(out.cpu(), dim = 1).tolist())
        y_true.extend(label)

report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names = enc.categories_[0])).T
report.astype({'support': int})

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
азиатский суп,0.500000,0.125000,0.200000,16
блины и оладьи,0.950072,0.950760,0.950416,1381
бургер,0.962500,0.947530,0.954957,5851
вегетарианская еда,0.560201,0.385501,0.456714,869
вок и лапша,0.924112,0.956546,0.940049,5385
гарниры,0.863207,0.881383,0.872200,5699
драники,0.655172,0.655172,0.655172,29
закуски,0.791148,0.659672,0.719453,4146
каша и гранола,0.891304,0.640625,0.745455,64
кофе,0.460317,0.263039,0.334776,441


# Results

In [1]:
import pandas as pd

pd.DataFrame(
    {
        'LogReg fasttext mean no weights' : [0.632497, 0.502078, 0.529205],
        'LogReg fasttext mean class weights' : [0.556070, 0.819777, 0.606949],
        'Naive Bayes CountVectorizer 100 words threshold' : [0.625416, 0.665721, 0.632699],
        'LogReg Tf-idf' : [0.715430, 0.663825, 0.673679],
        'biGRU max 30 sent len' : [0.831924, 0.768638, 0.785555],
        'CNN 1,2,3 kernel, max 30 sent len' : [0.812518, 0.770949, 0.785609],
        'biLSTM max 30 sent len': [0.810185, 0.801872, 0.801475],
        'biLSTM max 40 sent len + unfreeze emb' : [0.827857, 0.812538, 0.813990],

    }
    , index = ['precision', 'recall', 'f1-score']
).T

,precision,recall,f1-score
LogReg fasttext mean no weights,0.632497,0.502078,0.529205
LogReg fasttext mean class weights,0.556070,0.819777,0.606949
Naive Bayes CountVectorizer 100 words threshold,0.625416,0.665721,0.632699
LogReg Tf-idf,0.715430,0.663825,0.673679
biGRU max 30 sent len,0.831924,0.768638,0.785555
"CNN 1,2,3 kernel, max 30 sent len",0.812518,0.770949,0.785609
biLSTM max 30 sent len,0.810185,0.801872,0.801475
biLSTM max 40 sent len + unfreeze emb,0.827857,0.812538,0.813990


# Speller


In [ ]:
%cd /content/drive/MyDrive/NLP Made

/content/drive/MyDrive/NLP Made


In [ ]:
!pip install pyaspeller -q

In [ ]:
import numpy as np
to_spell = np.load('to_spell.npy').tolist()

In [ ]:
len(to_spell)

98719

In [ ]:
from pyaspeller import YandexSpeller
speller = YandexSpeller()

In [ ]:
speller.spelled('директорчкая')

'директорская'

In [ ]:
spelled = speller.spelled(' '.join(to_spell[98709:])).split()
print(len(spelled))

10


In [ ]:
to_spell[98709:]

['директорчкая',
 'пастичио',
 'пармариво',
 'фелицио',
 'ротату',
 'филицио',
 'емельяновский',
 'мачанкой',
 'тайбола',
 'бефсторгановым']

In [ ]:
final = list()

In [ ]:
final.extend(spelled)

In [ ]:
for i, j in zip(to_spell[98709:], final):
    if i != j:
        print(i, j)

директорчкая директорская


In [ ]:
len(speller.spelled(' '.join(to_spell[:10000])).split())

10000

In [ ]:
spelled

In [ ]:
import pandas as pd
import html
pd.set_option('display.max_rows', 500)

data = pd.read_pickle('menu_df_for_made.pkl')
data.dropna(inplace = True)
data['name_dish'] = data.name_dish.apply(lambda x: normalize(html.unescape(x)))
data['product_description'] = data.product_description.apply(lambda x: normalize(html.unescape(x)))

In [ ]:
len(data.product_description.unique())

811483

In [ ]:
to_spell = {i:None for i in to_spell}

In [ ]:
np.save('to_spell', to_spell)

In [ ]:
from threading import Thread
import numpy as np
from pyaspeller import YandexSpeller

def replace(subset):
    speller = YandexSpeller()

    for word in subset:
        correct = speller.spelled(word)
        to_spell[word] = correct

def split(a, n):
    k, m = divmod(len(a), n)
    return [a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]


def main(num_threads, num_words):
    # create threads
    work = list()
    subset_len = num_words*num_threads
    subset = [k for k, v in to_spell.items() if v is None][:subset_len]
    splitted = split(subset, num_threads)
    for item in splitted:
        work.append(item)
    threads = [Thread(target=replace, args=([job])) for job in work]

    # start the threads
    for thread in threads:
        thread.start()

    # wait for the threads to complete
    for thread in threads:
        thread.join()

    

if __name__ == "__main__":
    to_spell = np.load('to_spell.npy', allow_pickle=True).item()

    try:
        main(2,10)
    except Exception:
        np.save('to_spell', to_spell)
    
    np.save('to_spell', to_spell)

In [ ]:
%cd /content/drive/MyDrive/NLP Made
!pip install pyaspeller -q

/content/drive/MyDrive/NLP Made


In [ ]:
!python speller_threads.py

Completed 0 in Thread 0
Completed 0 in Thread 1
Completed 0 in Thread 2
Completed 0 in Thread 3
Completed 0 in Thread 4
Completed 0 in Thread 5
Completed 0 in Thread 6
Completed 0 in Thread 7
Completed 0 in Thread 8
Completed 0 in Thread 9
Completed 0 in Thread 10
Completed 0 in Thread 11
Completed 0 in Thread 12
Completed 0 in Thread 13
Completed 0 in Thread 14
Completed 0 in Thread 15
Completed 0 in Thread 16
Completed 0 in Thread 17
Completed 0 in Thread 18
Completed 0 in Thread 19
Completed 0 in Thread 20
Completed 0 in Thread 21
Completed 0 in Thread 22
Completed 0 in Thread 23
Completed 0 in Thread 24
Completed 0 in Thread 25
Completed 0 in Thread 26
Completed 0 in Thread 27
Completed 0 in Thread 28
Completed 0 in Thread 29
Completed 0 in Thread 30
Completed 0 in Thread 31
Completed 0 in Thread 32
Completed 0 in Thread 33
Completed 0 in Thread 34
Completed 0 in Thread 35
Completed 0 in Thread 36
Completed 0 in Thread 37
Completed 0 in Thread 38
Completed 0 in Thread 39
Finished T

In [ ]:
import numpy as np
to_spell = np.load('to_spell.npy', allow_pickle=True).item()

In [ ]:
len({k:v for k, v in to_spell.items() if v is not None})

0

In [ ]:
list({k:v for k, v in to_spell.items() if v is not None and k!=v}.items())[2500:2700]

[('рванные', 'рваные'),
 ('речпатый', 'репчатый'),
 ('куруруза', 'кукуруза'),
 ('покрошенного', 'покрашенного'),
 ('эдомер', 'эдамер'),
 ('гавядина', 'говядина'),
 ('моцареклла', 'моцарелла'),
 ('игредиент', 'ингредиент'),
 ('охтничьи', 'охотничьи'),
 ('коласки', 'коляски'),
 ('моцареллаx', 'моцарелла'),
 ('пепперониx', 'пепперони'),
 ('марджорио', 'марджори'),
 ('халап', 'халапеньо'),
 ('папмезан', 'пармезан'),
 ('ддрожжи', 'дрожжи'),
 ('сливи', 'сливки'),
 ('слабосоелный', 'слабосоленый'),
 ('мидиии', 'мидии'),
 ('чепотле', 'чипотле'),
 ('гарчичный', 'горчичный'),
 ('яйам', 'яйцам'),
 ('моцарелласыры', 'моцарелла сыры'),
 ('napolitano', 'наполитано'),
 ('томатычерри', 'томаты черри'),
 ('мяся', 'мяса'),
 ('колабски', 'колбаски'),
 ('телледжио', 'таледжио'),
 ('фиета', 'фиеста'),
 ('поидоры', 'помидоры'),
 ('регано', 'орегано'),
 ('оликовое', 'оливковое'),
 ('пармиджанно', 'пармиджанино'),
 ('невидалия', 'не видали'),
 ('пертрушка', 'петрушка'),
 ('макадами', 'макадамии'),
 ('пармезан

#Next

## Уникальных collected_name 45980

## Уникальных collected_description 41252

# После удаления опечаток

## Уникальных collected_name 35909

## Уникальных collected_description 28827

Для name_dish использовать пересечение двух collected_name и collected_description. Так можно избавиться от "оригинальных названий" типа Карлсон или Goods Game. Останется 20к слов. Хз

Для полученных словарей попробовать их отфильтровать получив векторы и прогнав через knn разделив на 2 класса. Но так могут потеряться слова, например поке -> лемма -> пока -> вектор не рядом с едой. Поэтому можно отдельно прогнать для лемм и оригинальных слов.
Хочется использовать фасттекст, потому что много слов порождено опечатками

Но, например, слово калифорния в нашем контексте, относится скорее к роллам, чем к штату, что может не заметить языковая модель. Поэтому популярные нужно оставить, а редкие проверить векторами

Можно прогнать на опечатки через яндекс спеллер

После очистки словарей отсортировать полученные значения в ячейках и оставить только уникальные строки



In [ ]:
from collections import Counter
Counter(collected_description).most_common()[-2400:-2000]

In [ ]:
model = gensim.models.fasttext.FastTextKeyedVectors.load('Fasttext/model.model')

In [ ]:
model.most_similar('ченсок')

[('дженсон', 0.5545827150344849),
 ('лоренсо', 0.5365540385246277),
 ('чень', 0.529872715473175),
 ('ченнай', 0.5085588693618774),
 ('мартинесс', 0.5060179829597473),
 ('ченгд', 0.501118540763855),
 ('тайпей', 0.49762624502182007),
 ('hongqiao', 0.49383875727653503),
 ('ухань', 0.487272709608078),
 ('чентаи', 0.4863824248313904)]

In [ ]:
!pip install pyaspeller

In [ ]:
from pyaspeller import YandexSpeller
speller = YandexSpeller()
fixed = speller.spelled('париезан')
fixed

'пармезан'